# Classification -> Pytorch
- Target
    1. build up sample RNN (construct layer)
    2. optimize RNN 

### Import libraries

In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

### Create mock data and plot

In [45]:
n_data = torch.ones(100,2)         # data base numbers 100x2 matrix 

x0 = torch.normal(2*n_data , 1)    # data point [zero cluster], normal(mean , std)     shape=(100,2)
y0 = torch.zeros(100)              # label      [zero cluster]                         shape=(100,1)

x1 = torch.normal(-2*n_data , 1)   # data point [ones cluster], normal(mean , std)     shape=(100,2)
y1 = torch.ones(100)               # label      [ones cluster]                         shape=(100,1)

# concat x data point 
x = torch.cat((x0 , x1) , 0).type(torch.FloatTensor)    # FloatTensor = 32-bit         shape=(200,2)
y = torch.cat((y0 , y1) , ).type(torch.LongTensor)      # LongTensor  = 64-bit integer shape=(200,1) (label 一定是要是 LongTensor format) 

### Building Classification Neural Network

In [48]:
# Net inherit torch.nn.Module
class Net(torch.nn.Module):
    # 定義 layer function (hidden)
    # n_feature -> 數據個數 , n_hidden -> hidden layer 神經元個數 , n_output -> output個數
    def __init__(self , n_feature , n_hidden , n_output):
        super(Net,self).__init__()
        # define hidden layer -> Linear function (numbers of input , numbers of output)
        # hidden layer = function(n_input , n_output)
        self.hidden =  torch.nn.Linear(n_feature , n_hidden)
        # output layer = function(n_input , n_output)
        self.predict = torch.nn.Linear(n_hidden , n_output)

    # 向前傳遞 -> 拿Layer來£用
    # x -> data
    def forward(self ,x):
        # Activated Function [F.relu] ( hidden layer(x[n_feature]) ) -> n_hidden
        x = F.relu(self.hidden(x))
        # self.predict(x[n_hidden]) -> n_output
        x = self.predict(x)
        return x   

# feature (x,y) -> input == 2 , output == 2
net = Net(2,10,2)   
print(net)

Net(
  (hidden): Linear(in_features=2, out_features=10, bias=True)
  (predict): Linear(in_features=10, out_features=2, bias=True)
)


### Optimize Neural Network 

In [ ]:
# torch.optim.{optimizer}({parameter})
# 優化神經網絡參數 -> 傳入神經網絡參數 , learning_rate 
optimizer = torch.optim.SGD(net.parameters() , lr=0.02)

# loss_function (CrossEntropyLoss 多分類 Loss_func problem)
loss_func = torch.nn.CrossEntropyLoss()   # 概率 [0.1 ,0.2 ,0.7] -> 相加 == 1 , 第三個分類最有機會

for t in range(100):
    out = net(x)

    #loss_func(prediction , real_value) 
    loss = loss_func(out , y)

    # 先把神經網絡裡面的參數的梯度 -> 0
    optimizer.zero_grad()
    # 誤差反向傳遞 loss => Variable (傳遞給每個節點這次計算出來的梯度)
    loss.backward()
    # 透過 optimizer 優化這些梯度
    optimizer.step()